In [1542]:
import pennylane as qml
import numpy as np
import tensorflow as tf

In [1543]:
dev = qml.device('default.qubit', wires=5)

In [1544]:
def real(angles, **kwargs):
    qml.Hadamard(wires=0)
    qml.RY(np.pi / 4, wires=1)
    qml.RX(np.pi / 4, wires=0)
    qml.CNOT(wires=[0, 1])
    qml.Rot(*angles, wires=0)
    qml.CNOT(wires=[0, 1])

In [1545]:
def generator(w, wires, **kwargs):
    qml.Hadamard(wires=wires[0])
    qml.RX(w[0], wires=wires[0])
    qml.RX(w[1], wires=wires[1])
    qml.RY(w[2], wires=wires[0])
    qml.RY(w[3], wires=wires[1])
    qml.RZ(w[4], wires=wires[0])
    qml.RZ(w[5], wires=wires[1])
    qml.CNOT(wires=[wires[0], wires[1]])
    qml.RX(w[6], wires=wires[0])
    qml.RY(w[7], wires=wires[0])
    qml.RZ(w[8], wires=wires[0])

In [1546]:
def utheta(theta, wires, **kwargs):
    qml.Hadamard(wires=wires[0])
    qml.RX(theta[0], wires=wires[0])
    qml.RY(theta[2], wires=wires[0])
    qml.RZ(theta[4], wires=wires[0])

In [1547]:
@qml.qnode(dev, interface="tf")
def qpca_train(angles, theta_weights, **kwargs):
    real(angles)
    utheta(theta_weights, [0, 1])
    z0 = qml.expval(qml.PauliZ(0))
    z1 = qml.expval(qml.PauliZ(1))
    return z0, z1

In [1548]:
@qml.qnode(dev, interface="tf")
def qpca(angles, theta_weights, **kwargs):
    real(angles)
    utheta(theta_weights, [0, 1])
    z0 = qml.expval(qml.PauliX(0))
    z1 = qml.expval(qml.PauliX(1))
    return z0, z1

In [1549]:
def pcaloss(angles, theta_weights, **kwargs):
    z0, z1 = qpca_train(angles, theta_weights)
    return 0.5 * ((z0 + 1) + 2 * (z1 + 1))

In [1550]:
angles = [np.pi / 6, np.pi / 2, np.pi / 7]

np.random.seed(0)
eps = 1e-2
init_theta_weights = np.array([np.pi] + [0] * 8) + \
                   np.random.normal(scale=eps, size=(9,))
theta_weights = tf.Variable(init_theta_weights)

opt = tf.keras.optimizers.SGD(0.4)

In [1551]:
cost = lambda: pcaloss(angles, theta_weights)
for step in range(200):
    opt.minimize(cost, theta_weights)
    if step % 5 == 0:
        cost_val = cost().numpy()
        print("Step {}: cost = {}".format(step, cost_val))

Step 0: cost = 1.5571886829117703
Step 5: cost = 1.2892331829779142
Step 10: cost = 1.1892890937754075
Step 15: cost = 1.1626307870074803
Step 20: cost = 1.1477379616540317
Step 25: cost = 1.134807694619431
Step 30: cost = 1.1240492155845228
Step 35: cost = 1.1162338778316263
Step 40: cost = 1.1112059428219005
Step 45: cost = 1.1082432576651444
Step 50: cost = 1.106592898182944
Step 55: cost = 1.1057032241145048
Step 60: cost = 1.1052320852196984
Step 65: cost = 1.1049848589500617
Step 70: cost = 1.1048557117153273
Step 75: cost = 1.1047883918636328
Step 80: cost = 1.104753335539718
Step 85: cost = 1.1047350886051888
Step 90: cost = 1.1047255930003252
Step 95: cost = 1.1047206520070627
Step 100: cost = 1.1047180810943749
Step 105: cost = 1.1047167434150966
Step 110: cost = 1.1047160474092899
Step 115: cost = 1.1047156852730495
Step 120: cost = 1.1047154968516097
Step 125: cost = 1.1047153988150025
Step 130: cost = 1.1047153478060898
Step 135: cost = 1.104715321265916
Step 140: cost = 1

In [1552]:
@qml.qnode(dev, interface="tf")
def get_real_prob(angles, theta_weights, **kwargs):
    real(angles)
    utheta(theta_weights, [0, 1])
    return qml.probs(wires=[0])


In [1553]:
prob_real = get_real_prob(angles, theta_weights)
purity_real = np.linalg.norm(prob_real) ** 2
print(prob_real)
print(purity_real)

tf.Tensor([0.10471529 0.89528471], shape=(2,), dtype=float64)
0.8124999999893755


In [1554]:
@qml.qnode(dev, interface="tf")
def get_gen_prob(gen_weights, theta_weights, **kwargs):
    generator(gen_weights, [2, 3])
    utheta(theta_weights, [2, 3])
    return qml.probs(wires=[2])

In [1555]:
@qml.qnode(dev, interface="tf")
def get_gen_purity(gen_weights, theta_weights, **kwargs):
    generator(gen_weights, [0, 1])
    utheta(theta_weights, [0, 1])
    generator(gen_weights, [2, 3])
    utheta(theta_weights, [2, 3])
    qml.Hadamard(wires=4)
    qml.CSWAP(wires=[4, 0, 2])
    qml.Hadamard(wires=4)
    return qml.expval(qml.PauliZ(4))


In [1556]:
init_gen_weights = np.array([np.pi] + [0] * 8) + \
                   np.random.normal(scale=eps, size=(9,))
gen_weights = tf.Variable(init_gen_weights)

In [1557]:
def gen_loss(gen_weights, theta_weights):
    global prob_real
    prob_gen = get_gen_prob(gen_weights, theta_weights)
    purity_gen = get_gen_purity(gen_weights, theta_weights)
    return - tf.reduce_mean(tf.reduce_sum(prob_real*tf.math.log(prob_gen))) - (purity_real * tf.math.log(purity_gen) + (1 - purity_real) * tf.math.log(1 - purity_gen))
    #return tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits = prob_gen, labels = prob_real))

In [1558]:
cost = lambda: gen_loss(gen_weights, theta_weights)
for step in range(100):
    opt.minimize(cost, gen_weights)
    if step % 5 == 0:
        cost_val = cost().numpy()
        print("Step {}: cost = {}".format(step, cost_val))

Step 0: cost = 1.1889426419156517
Step 5: cost = 0.8223649740647803
Step 10: cost = 0.8201229644474415
Step 15: cost = 0.8192186012793075
Step 20: cost = 0.818769626252056
Step 25: cost = 0.8185154843292599
Step 30: cost = 0.8183580655992153
Step 35: cost = 0.8182539494870826
Step 40: cost = 0.8181815685438522
Step 45: cost = 0.8181292410412724
Step 50: cost = 0.8180901985519654
Step 55: cost = 0.8180603019241094
Step 60: cost = 0.8180369054725596
Step 65: cost = 0.8180182545614646
Step 70: cost = 0.8180031485611776
Step 75: cost = 0.8179907435957373
Step 80: cost = 0.8179804325744128
Step 85: cost = 0.8179717697590043
Step 90: cost = 0.8179644219352906
Step 95: cost = 0.8179581359857468


In [1559]:
@qml.qnode(dev, interface="tf")
def show_real_diag(angles, theta_weights, **kwargs):
    real(angles)
    utheta(theta_weights, [0, 1])
    return qml.density_matrix([0])

@qml.qnode(dev, interface="tf")
def show_gen_diag(gen_weights, theta_weights, **kwargs):
    generator(gen_weights, [2, 3])
    utheta(theta_weights, [2, 3])
    return qml.density_matrix([2])
    
print(show_real_diag(angles, theta_weights))
print(show_gen_diag(gen_weights, theta_weights))

tf.Tensor(
[[ 1.04715292e-01+0.00000000e+00j -4.30110765e-08-2.30431119e-06j]
 [-4.30110765e-08+2.30431119e-06j  8.95284708e-01+0.00000000e+00j]], shape=(2, 2), dtype=complex128)
tf.Tensor(
[[ 0.10724913+0.j         -0.05171156-0.02397021j]
 [-0.05171156+0.02397021j  0.89275087+0.j        ]], shape=(2, 2), dtype=complex128)


In [1560]:
@qml.qnode(dev, interface="tf")
def show_real(angles):
    real(angles)
    return qml.density_matrix([0])

@qml.qnode(dev, interface="tf")
def show_gen(gen_weights):
    generator(gen_weights, [2, 3])
    return qml.density_matrix([2])
    
print(show_real(angles))
print(show_gen(gen_weights))

tf.Tensor(
[[ 0.19381378+0.j         -0.10847093-0.22524222j]
 [-0.10847093+0.22524222j  0.80618622+0.j        ]], shape=(2, 2), dtype=complex128)
tf.Tensor(
[[ 0.16336004+0.j         -0.11301068-0.17721179j]
 [-0.11301068+0.17721179j  0.83663996+0.j        ]], shape=(2, 2), dtype=complex128)


In [1561]:
obs = [qml.PauliX(0), qml.PauliY(0), qml.PauliZ(0)]

bloch_vector_real = qml.map(real, obs, dev, interface="tf")
bloch_vector_generator = qml.map(generator, obs, dev, interface="tf")

print("Real Bloch vector: {}".format(bloch_vector_real(angles)))
print("Generator Bloch vector: {}".format(bloch_vector_generator(gen_weights)))

Real Bloch vector: [-0.21694187  0.45048443 -0.61237244]
Generator Bloch vector: [-0.22602137  0.35442359 -0.67327992]
